In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {"user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.102 Yowser/2.5 Safari/537.36"}

auto_href_list = []
start = 1
finish = 100

with open('auto_hrefs_data.txt', 'w', encoding="utf-8") as file:
    while start < finish:
        try:
            for page_num in range(start, finish):
                url1 = 'https://moscow.drom.ru/auto/used/all/page'
                url2 = '/?frametype[]=10&frametype[]=1&frametype[]=11&distance=100'
                url = url1 + str(page_num) + url2
                response = requests.get(url, headers = headers)
                soup = BeautifulSoup(response.text, 'lxml')

                divs =  soup.find("div", class_="css-1nvf6xk eojktn00")

                hrefs = divs.find_all('a')

                for auto in hrefs:

                    auto_href = auto.get('href')
                    if 'auto/used' not in auto_href:
                        print(auto_href)
                        file.write(auto_href + '\n')

                if page_num >= finish - 1:
                    start = finish
        except:
            print(url)
            start = page_num
            if page_num >= finish - 1:
                start = finish

In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {"user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.102 Yowser/2.5 Safari/537.36"}

auto_href_list = []
start = 0

with open('auto_hrefs_data.txt', 'r') as file:
    auto_hrefs = file.readlines()


finish = len(auto_hrefs)
    
with open('auto_data.csv', 'w', encoding="cp1251") as file:
    file.write('AutoName;Year;AutoURL;Price;Fuel;EngineCapacity;HorsePower;Mileage')
    while start < finish:
        try:
            for href_num in range(start, finish):

                url = auto_hrefs[href_num][:-1]
                response_auto = requests.get(url, headers = headers)
                soup_auto = BeautifulSoup(response_auto.text, 'lxml')
                
                auto_data = []
                div_price =  soup_auto.find("div", class_="css-eazmxc e162wx9x0")
                if div_price is not None:
                    price_l = div_price.get_text().split()
                    price = ''.join(price_l[:-1])

                    # имя
                    auto_name =  soup_auto.find("span", class_="css-1kb7l9z e162wx9x0")
                    auto_name_list = auto_name.get_text().split(', ')
                    auto_data.append(auto_name_list[0].replace('Продажа ', ''))

                    # Год    
                    auto_data.append(auto_name_list[1][:4])

                    # Ссылка
                    auto_data.append(url)

                    # Цена
                    auto_data.append(price)

                    table_TTX =  soup_auto.find("table", class_="css-xalqz7 eppj3wm0")
                    table_TTX_body = table_TTX.tbody

                    # Двигатель
                    dviglo = table_TTX_body.find("span", class_="css-1jygg09 e162wx9x0").get_text()

                    ## тип топлива
                    dviglo_list = dviglo.split(', ')
                    auto_data.append(dviglo_list[0])

                    # Мощность
                    power = table_TTX_body.find("span", class_="css-9g0qum e162wx9x0").get_text()
                    auto_data.append(power.split()[0])

                    # Пробег
                    probeg = table_TTX_body.find("span", class_="css-1osyw3j ei6iaw00").get_text()
                    auto_data.append(''.join(probeg.split()))

                    auto_data1 = ';'.join(auto_data)
                    
                    file.write(auto_data1 + '\n')

                if href_num >= finish - 1:
                    start = finish
        except:
            print(url)
            start = href_num +1
            if href_num >= finish - 1:
                start = finish